In [ ]:
import pandas as pd
from datetime import datetime as dt
from copy import deepcopy
from itertools import product
import tensorflow as tf
import numpy as np
from collections import deque
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
def run_model(category):
  df = pd.read_csv('./orders.csv')
  df = df[df.category == category]

  orders = df['orders']
  values = orders.values
  training_data_len = math.ceil(len(values)* 0.8)

  scaler = MinMaxScaler(feature_range=(0,1))
  scaled_data = scaler.fit_transform(values.reshape(-1,1))
  train_data = scaled_data[0: training_data_len, :]

  x_train = []
  y_train = []

  for i in range(3, len(train_data)):
      x_train.append(train_data[i-3:i, 0])
      y_train.append(train_data[i, 0])

  x_train, y_train = np.array(x_train), np.array(y_train)
  x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

  test_data = scaled_data[training_data_len-3: , : ]
  x_test = []
  y_test = values[training_data_len:]

  for i in range(3, len(test_data)):
    x_test.append(test_data[i-3:i, 0])

  x_test = np.array(x_test)
  x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

  # make model
  model = Sequential()
  model.add(LSTM(40, return_sequences=True, input_shape=(x_train.shape[1], 1)))
  model.add(Dense(40))
  model.add(LSTM(20, return_sequences=False))
  model.add(Dense(20))
  model.add(Dense(20))
  model.add(Dense(1))

  # train model
  model.compile(optimizer='adam', loss='mean_squared_error')
  print(model.summary())
  model.fit(x_train, y_train, batch_size=1, epochs=150, verbose=False)

  # calculate predictions
  training_predictions = model.predict(x_train)
  training_predictions = scaler.inverse_transform(training_predictions)[:,0]
  predictions = model.predict(x_test)
  predictions = scaler.inverse_transform(predictions)[:,0]
  rmse = np.sqrt(np.mean(predictions - y_test)**2)

  # make plot
  train = pd.DataFrame({'orders': scaler.inverse_transform(y_train.reshape(-1, 1))[:,0].tolist(), 'predictions': training_predictions.tolist()})
  test = pd.DataFrame({'orders': y_test.tolist(), 'predictions': predictions.tolist()})
  fig, axs = plt.subplots(1, 2, figsize=(12, 5))
  fig.suptitle(category, fontsize='large')
  axs[0].plot(range(1, len(train) + 1), train.orders, label='actual')
  axs[0].plot(range(1, len(train) + 1), train.predictions, label='predictions')
  axs[0].set_xlabel('Weeks')
  axs[0].set_ylabel('Orders')
  axs[0].set_title('Training')
  axs[0].legend()
  axs[1].plot(range(1, len(test) + 1), test.orders, label='actual')
  axs[1].plot(range(1, len(test) + 1), test.predictions, label='predictions')
  axs[1].legend()
  axs[1].set_xlabel('Weeks')
  axs[1].set_ylabel('Orders')
  axs[1].set_title('Testing')
  plt.show()

  return rmse

In [ ]:
run_model('bed_bath_table')

In [ ]:
run_model('health_beauty')

In [ ]:
run_model('sports_leisure')

In [ ]:
run_model('furniture_decor')

In [ ]:
run_model('computers_accessories')